# How to load huge CSV datasets in Python Pandas

It may happen that you have a huge CSV dataset which occupies 4 or 5 GBytes (or even more) in your hard disk and you want to process it with Python `pandas`. Maybe you don't need all the data contained in the dataset, but only some records satisfying some criteria.
In this short tutorial I show you how to deal with huge datasets in Python `Pandas`.

We can apply four strategies:
* vertical filter
* horizontal filter
* bursts
* memory.

## Vertical Filter
In this case we load only some columns of the whole dataset. We can use the parameter `usecols` of the `read_csv()` function to select only some columns. 

In [35]:
import pandas as pd
df = pd.read_csv('hepatitis.csv', usecols=['age','sex'])
df.head()

,age,sex
0,30,male
1,50,female
2,78,female
3,31,female
4,34,female


## Horizontal Filter
In this case we load only some rows of the dataset. We can choose the starting row and how many rows we must load. Keep in mind that if we skip the first row, we must pass the names of the columns as further parameter.

In [36]:
srow = 5
nrows = 50
columns = ['age','sex','steroid','antivirals','fatigue','malaise','anorexia','liver_big','liver_firm','spleen_palpable','spiders','ascites','varices','bilirubin','alk_phosphate','sgot','albumin','protime','histology','class']
df = pd.read_csv('hepatitis.csv', skiprows=srow, nrows=nrows, names=columns)
df.head()

,age,sex,steroid,antivirals,fatigue,malaise,anorexia,liver_big,liver_firm,spleen_palpable,spiders,ascites,varices,bilirubin,alk_phosphate,sgot,albumin,protime,histology,class
0,34,female,True,False,False,False,False,True,False,False,False,False,False,1.0,NaN,200.0,4.0,NaN,False,live
1,34,female,True,False,False,False,False,True,False,False,False,False,False,0.9,95.0,28.0,4.0,75.0,False,live
2,51,female,False,False,True,False,True,True,False,True,True,False,False,NaN,NaN,NaN,NaN,NaN,False,die
3,23,female,True,False,False,False,False,True,False,False,False,False,False,1.0,NaN,NaN,NaN,NaN,False,live
4,39,female,True,False,True,False,False,True,True,False,False,False,False,0.7,NaN,48.0,4.4,NaN,False,live


## Bursts
It may happen that we need to load the full dataset, but we don't have enough memory to load it. Thus we can load it in bursts, then we apply some filters and store the results into another (smaller) dataset. Filters may include dropping operations and conversion from bigger to smaller types.

Firstly, we define a function called `read_burst()` which reads the i-th burst, performs some filters and then stores the results into another output csv file. In our case we can drop rows where `malaise = False`. We pass the starting row `srow`, the burst size `burst` the names of columns `columns` as parameters of the `read_burst()` function. When the starting row is equal to ,1 we must also write the header to the output csv file. Thus we define a variable called `header`, which is set to `True` if `srow = 1`.

In [37]:
def read_burst(srow,burst,columns):
    header = False
    if srow == 1:
        header = True
    df = pd.read_csv('hepatitis.csv', skiprows=srow, nrows=burst, names=columns)
    df = df.drop(df[df['malaise'] == False].index)
    df.to_csv('hepatitis_small.csv', mode='a',header=header)

Now we can loop throughout the dataset. Note that we must know in advance the number of lines of the csv file. This can be done by using a command line tick. We can use the `wc` unix shell command. In order to run this command within the jupyther notebook, we must use the `!` operator.

In [38]:
! wc -l hepatitis.csv

     156 hepatitis.csv


Our file contains 156 rows, thus we can set the maximum number of lines to be read to 156, since the first line corresponds to the header. We set `burst = 10`. Thus we read 100 at times. At the end, we run again the `read_burst()` function to load the last remaining rows.

In [39]:
burst =10
srow = 1
nrows = 156
columns = ['age','sex','steroid','antivirals','fatigue','malaise','anorexia','liver_big','liver_firm','spleen_palpable','spiders','ascites','varices','bilirubin','alk_phosphate','sgot','albumin','protime','histology','class']
while srow < nrows:
    print('srow: ' + str(srow))
    read_burst(srow,burst,columns)
    srow = srow + burst

read_burst(srow,nrows,columns)  

srow: 1
srow: 11
srow: 21
srow: 31
srow: 41
srow: 51
srow: 61
srow: 71
srow: 81
srow: 91
srow: 101
srow: 111
srow: 121
srow: 131
srow: 141
srow: 151


Now we can load the smaller dataset.

In [53]:
df = pd.read_csv('hepatitis_small.csv')
df.head()

,Unnamed: 0,age,sex,steroid,antivirals,fatigue,malaise,anorexia,liver_big,liver_firm,...,spiders,ascites,varices,bilirubin,alk_phosphate,sgot,albumin,protime,histology,class
0,5,38,female,False,False,True,True,True,True,False,...,False,True,False,2.0,72.0,89.0,2.9,46.0,False,live
1,1,27,female,True,False,True,True,True,False,True,...,True,False,False,1.2,133.0,98.0,4.1,39.0,False,live
2,5,27,female,False,False,True,True,False,True,False,...,False,False,False,0.8,95.0,46.0,3.8,100.0,False,live
3,6,49,female,False,True,True,True,True,True,True,...,True,False,False,0.6,85.0,48.0,3.7,NaN,False,live
4,9,51,female,False,True,True,True,False,True,False,...,False,False,False,1.0,78.0,58.0,4.6,52.0,False,live


We can drop the `Unnamed: 0` column.

In [54]:
df.drop(columns=['Unnamed: 0'],axis=1,inplace=True)
df.head()

,age,sex,steroid,antivirals,fatigue,malaise,anorexia,liver_big,liver_firm,spleen_palpable,spiders,ascites,varices,bilirubin,alk_phosphate,sgot,albumin,protime,histology,class
0,38,female,False,False,True,True,True,True,False,False,False,True,False,2.0,72.0,89.0,2.9,46.0,False,live
1,27,female,True,False,True,True,True,False,True,True,True,False,False,1.2,133.0,98.0,4.1,39.0,False,live
2,27,female,False,False,True,True,False,True,False,False,False,False,False,0.8,95.0,46.0,3.8,100.0,False,live
3,49,female,False,True,True,True,True,True,True,False,True,False,False,0.6,85.0,48.0,3.7,NaN,False,live
4,51,female,False,True,True,True,False,True,False,False,False,False,False,1.0,78.0,58.0,4.6,52.0,False,live


# Memory Configuration
Another option while reading huge datasets in Python `pandas` could be increasing the memory associated to the reading operation. This can be done through the `low_memory` parameter.

In [47]:
df = pd.read_csv('hepatitis.csv', low_memory=False)
df.head()

,age,sex,steroid,antivirals,fatigue,malaise,anorexia,liver_big,liver_firm,spleen_palpable,spiders,ascites,varices,bilirubin,alk_phosphate,sgot,albumin,protime,histology,class
0,30,male,False,False,False,False,False,False,False,False,False,False,False,1.0,85.0,18.0,4.0,NaN,False,live
1,50,female,False,False,True,False,False,False,False,False,False,False,False,0.9,135.0,42.0,3.5,NaN,False,live
2,78,female,True,False,True,False,False,True,False,False,False,False,False,0.7,96.0,32.0,4.0,NaN,False,live
3,31,female,NaN,True,False,False,False,True,False,False,False,False,False,0.7,46.0,52.0,4.0,80.0,False,live
4,34,female,True,False,False,False,False,True,False,False,False,False,False,1.0,NaN,200.0,4.0,NaN,False,live
